In [1]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("No GPU/MPS available, falling back to CPU.")

Using GPU


In [2]:
""" from mesh_video_generator import MeshVideoGenerator

hw = 32
num_views = 2
data_dirs = ["meshes", "SHREC19_MH_dataset", "SHREC20b_hires/models", "MPI-FAUST/training/scans"]

generator = MeshVideoGenerator(
    output_dir=data_dirs[0],
    hw=hw,
    num_views=num_views,
    use_normal_map=True,
    device=device
) """
#generator.process_folder("meshes", display_frames=False)
#generator.process_single_mesh("meshes", index=5)

' from mesh_video_generator import MeshVideoGenerator\n\nhw = 32\nnum_views = 2\ndata_dirs = ["meshes", "SHREC19_MH_dataset", "SHREC20b_hires/models", "MPI-FAUST/training/scans"]\n\ngenerator = MeshVideoGenerator(\n    output_dir=data_dirs[0],\n    hw=hw,\n    num_views=num_views,\n    use_normal_map=True,\n    device=device\n) '

In [3]:
#import torch
from diff3f import get_features_per_vertex
from diffusion import init_pipe

from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors
from dataloaders.mesh_container import MeshContainer

from sam2_setup import init_sam2
#from functional_map import compute_surface_map

In [4]:
num_views = 49
H = 512
W = 512
num_images_per_prompt = 1
tolerance = 0.004
random_seed = 42
use_normal_map = True
is_tosca = False

In [5]:
def compute_features(device, sam_model,pipe, m, prompt, num_views, H, W, tolerance=0.01):
    mesh = convert_mesh_container_to_torch_mesh(m, device=device, is_tosca=False)
    mesh_vertices = mesh.verts_list()[0]

    features = get_features_per_vertex(
        device=device,
        sam_model=sam_model,
        pipe=pipe,
        mesh=mesh,
        prompt=prompt,
        mesh_vertices=mesh_vertices,
        num_views=num_views,
        H=H,
        W=W,
        tolerance=tolerance
    )
    return features.cpu()

In [6]:
sam_model = init_sam2(device)
pipe = None


In [7]:
source_file_path = "meshes/cow.obj"
target_file_path = "meshes/camel.obj"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)

In [8]:
f_source = compute_features(device, sam_model, pipe, source_mesh, "cow", num_views, H, W)
f_target = compute_features(device, sam_model, pipe, target_mesh, "camel", num_views, H, W)

Starting batch_render with num_views=49, H=512, W=512
Rendering completed successfully
Starting batch_render with num_views=49, H=512, W=512
Rendering completed successfully


/workspace/miniconda3/envs/diff3f/lib/python3.10/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1729805341246/work/aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 98/98 [00:41<00:00,  2.38it/s]


Number of missing features:  1
Copied features from nearest vertices
Time taken in mins:  0.8680500586827596
Starting batch_render with num_views=49, H=512, W=512
Rendering completed successfully
Starting batch_render with num_views=49, H=512, W=512
Rendering completed successfully


100%|██████████| 98/98 [00:38<00:00,  2.53it/s]


Number of missing features:  0
Copied features from nearest vertices
Time taken in mins:  0.771443772315979


In [9]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  